Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup as bsoup

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
raw_random_wikipedia_page = requests.get(wikipedia_link)

In [5]:
page = raw_random_wikipedia_page.text
#print(page)

In [6]:
soup = bsoup(page, "lxml")
tablePostalCode = soup.find_all("table")[0]
rows = tablePostalCode.find_all("tr")
listPostalCode = []
for row in rows:
    tds = row.find_all("td")
    for td in tds:
        if (td.p.span.find("i") is not None) and ("Not assigned" in td.p.span.i.text):
            continue
        spanText = td.p.span.text
        code = td.p.b.text
        aTags = td.find_all("a")
        if len(aTags) <= 0:
            continue
        borough = aTags[0].text
        listNeighborhood = []
        if len(aTags) > 1:
            neighborhood = spanText[spanText.find("(") + 1: len(spanText) - 1]
        else:
            neighborhood = borough
        listPostalCode.append({"PostalCode": code, "Borough": borough, "Neighborhood": neighborhood})
listPostalCode

[{'Borough': 'North York', 'Neighborhood': 'Parkwoods', 'PostalCode': 'M3A'},
 {'Borough': 'North York',
  'Neighborhood': 'Victoria Village',
  'PostalCode': 'M4A'},
 {'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park / Harbourfront',
  'PostalCode': 'M5A'},
 {'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor / Lawrence Heights',
  'PostalCode': 'M6A'},
 {'Borough': "Queen's Park",
  'Neighborhood': "Queen's Park",
  'PostalCode': 'M7A'},
 {'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue',
  'PostalCode': 'M9A'},
 {'Borough': 'Scarborough',
  'Neighborhood': 'Malvern / Rouge',
  'PostalCode': 'M1B'},
 {'Borough': 'North York',
  'Neighborhood': 'Don Mills)Nort',
  'PostalCode': 'M3B'},
 {'Borough': 'East York',
  'Neighborhood': 'Parkview Hill / Woodbine Gardens',
  'PostalCode': 'M4B'},
 {'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson',
  'PostalCode': 'M5B'},
 {'Borough': 'North York', 'Neighborhood': 'North York', 'Postal

In [7]:
columns = ["PostalCode", "Borough", "Neighborhood"]
dfPostalCode = pd.DataFrame.from_records(data=listPostalCode, columns=columns)
dfPostalCode

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills)Nort
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Explain my work and any assumptions I made:
Firstly, I used etree lib from lxml to scrap Postal Code table content, then pd.read_html lib to convert html content to data frame as code below:

root = etree.XML(page)
tableTag = etree.XPath("//table")
postalCodeTorontoElems = tableTag(root)[0]
listDfTorontoPostalCode = pd.read_html(etree.tostring(postalCodeTorontoElems,method='html'))
listCol = list(listDfTorontoPostalCode[0].columns)
print(listDfTorontoPostalCode[0].shape)
dfTorontoPostalCode = listDfTorontoPostalCode[0]

I realized that there were some difficult cases to seperate which is borough or neighborhood, because the format/structure of cells in Wiki Postal Code table was not the same while all content in each cell of data frame is text and no spaces. For example, cell(1,7), cell(3,3), cell(2,8), cell(3,8).

Thus, I used etree lib to read and seperate each element that I was interested in. But it's not a good way.

Then, it's realy luck to me when I saw the note in this assignment. I used BeautifulSoup lib as recommendation in note of assignment. It's really easy and make my task on the fly!

I supposed that the first a tag is represented the borough, and all text in "()" is represented the list of neighborhoods in borough.

In [8]:
dfPostalCode.shape

(101, 3)